In [3]:
import pandas as pd
import http.client
import json
import csv
import os
from textblob import TextBlob
import numpy as np
import time
import gc
import random
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

In [4]:
def get_sentiment(text):
    sentDict = {}
    doc = TextBlob(text)
    sentDict['polarity'] = doc.sentiment.polarity
    sentDict['subjectivity'] = doc.sentiment.subjectivity
    return sentDict

In [5]:
post14 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2014.txt")
post15 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2015.txt")
post16 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2016.txt")
post17 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2017.txt")
post18 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2018.txt")
post19 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2019.txt")
post20 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2020.txt")
post21 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/post2021.txt")
frames = [post14,post15,post16,post17,post18,post19,post20,post21]
post = pd.concat(frames)

In [6]:
post['created_utc'] = pd.to_datetime(post['created_utc'])

In [7]:
post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844931 entries, 0 to 47199
Data columns (total 10 columns):
created_utc              844931 non-null datetime64[ns]
submissionID             844931 non-null object
full_link                844931 non-null object
author                   844931 non-null object
author_flair_richtext    844931 non-null object
num_comments             844931 non-null int64
score                    844931 non-null int64
title                    844931 non-null object
selftext                 844931 non-null object
link                     844931 non-null object
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 70.9+ MB


In [9]:
post.head()

,created_utc,submissionID,full_link,author,author_flair_richtext,num_comments,score,title,selftext,link
0,2014-12-31,2qzjqr,,livebitcoinnews,,0,1,"CheapAir now Taking over Bitcoin, Dogecoin and...",,https://www.reddit.com/r/Bitcoin/comments/2qzj...
1,2014-12-31,2qziky,,BitStashCTO,,6,30,"If ever Bitcoin had a use case, Venezuela is it !",,https://www.reddit.com/r/Bitcoin/comments/2qzi...
2,2014-12-31,2qziao,,SThist,,11,134,Police suspect only 1 percent taken by Mt. Gox...,,https://www.reddit.com/r/Bitcoin/comments/2qzi...
3,2014-12-31,2qzhov,,bitsforall,,1,0,2015 - WTF - Ben Lawsky did you miss your own ...,,https://www.reddit.com/r/Bitcoin/comments/2qzh...
4,2014-12-31,2qzh5h,,bittopia,,27,33,Do you guys stash dummy BTC wallets everywhere...,I have 10 wallets on a few PCs and mobile devi...,https://www.reddit.com/r/Bitcoin/comments/2qzh...


In [10]:
comment14 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2014.txt")
comment15 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2015.txt")
comment16 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2016.txt")
comment17 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2017.txt")
comment18 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2018.txt")
comment19 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2019.txt")
comment20 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2020.txt")
comment21 = pd.read_json("/Users/jonhuaqin/Downloads/Reddit/comments2021.txt")
frames = [comment14,comment15,comment16,comment17,comment18,comment19,comment20,comment21]
comment = pd.concat(frames)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [11]:
comment['created_utc'] = pd.to_datetime(comment['created_utc'])
comment["post_id"] = comment["link_id"].str.split("_").str[-1]

In [12]:
comment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2154200 entries, 0 to 62299
Data columns (total 12 columns):
author                   object
author_flair_richtext    object
body                     object
commentID                object
created_utc              datetime64[ns]
is_submitier             object
is_submitter             object
link                     object
link_id                  object
parent_id                object
score                    int64
post_id                  object
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 213.7+ MB


In [13]:
comment.head()

,author,author_flair_richtext,body,commentID,created_utc,is_submitier,is_submitter,link,link_id,parent_id,score,post_id
0,statoshi,,The direct incentive is to have your own trust...,cnaz5kk,2014-12-31,NaN,,,t3_2qygsn,t1_cnas5eg,8,2qygsn
1,throughnothing,,"Though as hardware wallets go, ledger has some...",cnaz3u1,2014-12-31,NaN,,,t3_2qxrnx,t1_cnaobhk,0,2qxrnx
2,empraptor,,I guess that's true in a sense.\n\nIt's just t...,cnaz3sc,2014-12-31,NaN,,,t3_2qx1sz,t1_cnad60o,1,2qx1sz
3,tulipfutures,,"Or you'll realize, perhaps in another year, th...",cnaz3s1,2014-12-31,NaN,,,t3_2qze2q,t1_cnayywz,-16,2qze2q
4,odal_limbs,,"For me, 2014 was the year where the idea that ...",cnaz3mm,2014-12-31,NaN,,,t3_2qz6vb,t1_cnayo12,10,2qz6vb


In [14]:
comment1 = comment.groupby('post_id')['author'].nunique().to_frame(name='Num_of_repliedids')
comment2 = comment.groupby('post_id').agg({'score': 'mean'})
comment3 = comment2[['score']].rename(columns={'score': 'comments_score'})
comment_s = comment1.merge(comment3, how='inner', left_on='post_id', right_on='post_id')

In [15]:
comment_s.head()

,Num_of_repliedids,comments_score
post_id,,
1gsd99,1,1.0
1im6hw,2,0.5
1j2voc,1,1.0
1j66ry,1,1.0
1j7ybq,1,1.0


In [16]:
post_all = post.merge(comment_s, how='inner', left_on='submissionID', right_on='post_id' )

In [88]:
post_all['next_date'] = pd.DatetimeIndex(post_all.created_utc) + pd.DateOffset(1)

In [89]:
post_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242004 entries, 0 to 242003
Data columns (total 13 columns):
created_utc              242004 non-null datetime64[ns]
submissionID             242004 non-null object
full_link                242004 non-null object
author                   242004 non-null object
author_flair_richtext    242004 non-null object
num_comments             242004 non-null int64
score                    242004 non-null int64
title                    242004 non-null object
selftext                 242004 non-null object
link                     242004 non-null object
Num_of_repliedids        242004 non-null int64
comments_score           242004 non-null float64
next_date                242004 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(7)
memory usage: 25.8+ MB


In [90]:
post_all.head()

,created_utc,submissionID,full_link,author,author_flair_richtext,num_comments,score,title,selftext,link,Num_of_repliedids,comments_score,next_date
0,2014-12-31,2qziky,,BitStashCTO,,6,30,"If ever Bitcoin had a use case, Venezuela is it !",,https://www.reddit.com/r/Bitcoin/comments/2qzi...,1,1.000000,2015-01-01
1,2014-12-31,2qziky,,BitStashCTO,,6,30,"If ever Bitcoin had a use case, Venezuela is it !",,https://www.reddit.com/r/Bitcoin/comments/2qzi...,1,1.000000,2015-01-01
2,2014-12-31,2qzh5h,,bittopia,,27,33,Do you guys stash dummy BTC wallets everywhere...,I have 10 wallets on a few PCs and mobile devi...,https://www.reddit.com/r/Bitcoin/comments/2qzh...,3,2.250000,2015-01-01
3,2014-12-31,2qzh5h,,bittopia,,27,33,Do you guys stash dummy BTC wallets everywhere...,I have 10 wallets on a few PCs and mobile devi...,https://www.reddit.com/r/Bitcoin/comments/2qzh...,3,2.250000,2015-01-01
4,2014-12-31,2qzfks,,btcRaleigh,,19,57,Random Bitcoin fact I learned today,Bitcoin addresses consist of random digits and...,https://www.reddit.com/r/Bitcoin/comments/2qzf...,6,1.285714,2015-01-01


In [19]:
post1 = post_all.groupby('created_utc')['submissionID'].nunique().to_frame(name='Num_of_submissionID')
post2 = post_all.groupby('created_utc')['author'].nunique().to_frame(name='Num_of_author')
post3 = post_all.groupby('created_utc').agg({'num_comments': 'sum', 'Num_of_repliedids': 'sum', 'score': 'mean', 'comments_score': 'mean'})


In [20]:
post3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2612 entries, 2013-12-31 to 2021-03-17
Data columns (total 4 columns):
num_comments         2612 non-null int64
Num_of_repliedids    2612 non-null int64
score                2612 non-null float64
comments_score       2612 non-null float64
dtypes: float64(2), int64(2)
memory usage: 102.0 KB


In [21]:
join = post1.merge(post2, how='inner', left_on='created_utc', right_on='created_utc')
post_s = join.merge(post3, how='inner', left_on='created_utc', right_on='created_utc')

In [22]:
post_s.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2612 entries, 2013-12-31 to 2021-03-17
Data columns (total 6 columns):
Num_of_submissionID    2612 non-null int64
Num_of_author          2612 non-null int64
num_comments           2612 non-null int64
Num_of_repliedids      2612 non-null int64
score                  2612 non-null float64
comments_score         2612 non-null float64
dtypes: float64(2), int64(4)
memory usage: 142.8 KB


In [23]:
post_s.head()

,Num_of_submissionID,Num_of_author,num_comments,Num_of_repliedids,score,comments_score
created_utc,,,,,,
2013-12-31,29,21,937,110,42.413793,2.178298
2014-01-01,74,60,2229,281,60.743243,1.463552
2014-01-02,115,92,4522,683,61.495652,1.980248
2014-01-03,121,99,3646,518,53.247934,1.998339
2014-01-04,141,115,3396,632,41.978723,1.826934


In [24]:
bitcoin = pd.read_csv('BTC-USD.csv')

In [25]:
bitcoin['delta'] = bitcoin['Adj Close'] - bitcoin['Open']
bitcoin['increase'] = bitcoin.apply(lambda x: 1 if x['delta']>0 else 0, axis=1)
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])


In [26]:
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2373 entries, 0 to 2372
Data columns (total 9 columns):
Date         2373 non-null datetime64[ns]
Open         2369 non-null float64
High         2369 non-null float64
Low          2369 non-null float64
Close        2369 non-null float64
Adj Close    2369 non-null float64
Volume       2369 non-null float64
delta        2369 non-null float64
increase     2373 non-null int64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 167.0 KB


In [27]:
bitcoin.head()

,Date,Open,High,Low,Close,Adj Close,Volume,delta,increase
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0,-8.529999,0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0,-32.419983,0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0,-29.307007,0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0,14.230988,1
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0,-9.263977,0


In [28]:
dataset_temp = post_s.merge(bitcoin, how='inner', left_on='created_utc', right_on='Date')
dataset = dataset_temp.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'delta' ],axis=1)

In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2351 entries, 0 to 2350
Data columns (total 7 columns):
Num_of_submissionID    2351 non-null int64
Num_of_author          2351 non-null int64
num_comments           2351 non-null int64
Num_of_repliedids      2351 non-null int64
score                  2351 non-null float64
comments_score         2351 non-null float64
increase               2351 non-null int64
dtypes: float64(2), int64(5)
memory usage: 146.9 KB


In [34]:
dataset.head()

,Num_of_submissionID,Num_of_author,num_comments,Num_of_repliedids,score,comments_score,increase
0,165,127,3133,900,35.381818,1.589498,0
1,80,67,2593,318,62.650000,1.285665,0
2,119,96,3288,622,37.941176,1.723893,0
3,90,73,2378,594,42.044444,2.053441,1
4,116,91,2536,932,34.344828,1.680739,0


In [35]:
x_data = dataset.loc[:, dataset.columns != "increase"]
y_data = dataset.loc[:, "increase"]

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, shuffle = True, random_state = 614)

In [43]:
#export
class LinearRegressionModel():
    
    # points [2]
    def linearClassifier(self,x_train, x_test, y_train):
        # TODO: Create a LinearRegression classifier and train it.
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        linear = LinearRegression().fit(x_train,y_train)
        y_predict_train = linear.predict(x_train)
        y_predict_test = linear.predict(x_test)
        
        # -------------------------------
        return y_predict_train, y_predict_test

    # points [1]
    def lgTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy (on the training set) using the accuracy_score method.
        # Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use any method that satisfies the requriements.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE 
        #y_predict_train_round = [round(k) for k in y_predict_train]
        for i in range(len(y_predict_train)):
            if y_predict_train[i] >= 0.5: y_predict_train[i] = 1
            else: y_predict_train[i] = 0
        train_accuracy = accuracy_score(y_train, y_predict_train)
 
        # -------------------------------   
        return train_accuracy
    
    # points [1]
    def lgTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy (on the testing set) using the accuracy_score method.
        # Note: Round the output values greater than or equal to 0.5 to 1 and those less than 0.5 to 0. You can use any method that satisfies the requriements.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        #y_predict_test_round = [round(k) for k in y_predict_test]
        for i in range(len(y_predict_test)):
            if y_predict_test[i] >= 0.5: y_predict_test[i] = 1
            else: y_predict_test[i] = 0
        test_accuracy = accuracy_score(y_test, y_predict_test)
        # -------------------------------
        return test_accuracy
    


In [44]:
Linear = LinearRegressionModel()


In [46]:
y_predict_train, y_predict_test = Linear.linearClassifier(x_train, x_test, y_train)

In [47]:
train_accuracy = Linear.lgTrainAccuracy(y_train,y_predict_train)

In [48]:
test_accuracy = Linear.lgTestAccuracy(y_test,y_predict_test)

In [50]:
print("train_accuracy = " + str(train_accuracy))
print("test_accuracy = " + str(test_accuracy))

train_accuracy = 0.5525835866261398
test_accuracy = 0.546742209631728


In [51]:
#export
class RFClassifier():
    
    # points [2]
    def randomForestClassifier(self,x_train,x_test, y_train):
        # TODO: Create a RandomForestClassifier and train it. Set Random state to 614.
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: RandomForestClassifier object, numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        rf_clf = RandomForestClassifier(random_state = 614).fit(x_train, y_train)
        y_predict_train = rf_clf.predict(x_train)
        y_predict_test = rf_clf.predict(x_test)
        # -------------------------------
        return rf_clf,y_predict_train, y_predict_test
    
    # points [1]
    def rfTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy on the training set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        train_accuracy = accuracy_score(y_train, y_predict_train)
        # -------------------------------
        return train_accuracy
    
    # points [1]
    def rfTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy on the test set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        test_accuracy = accuracy_score(y_test, y_predict_test)
        # -------------------------------
        return test_accuracy
    
# Q3.3.1 Feature Importance
    
    # points [1]
    def rfFeatureImportance(self,rf_clf):
        # TODO: Determine the feature importance as evaluated by the Random Forest Classifier.
        # args: RandomForestClassifier object
        # return: float array
        # -------------------------------
        # ADD CODE HERE
        
        feature_importance = rf_clf.feature_importances_
        # -------------------------------
        return feature_importance
    
    # points [1]
    def sortedRFFeatureImportanceIndicies(self,rf_clf):
        # TODO: Sort them in the ascending order and return the feature numbers[0 to ...].
        #       Hint: There is a direct function available in sklearn to achieve this. Also checkout argsort() function in Python.
        # args: RandomForestClassifier object
        # return: int array
        # -------------------------------
        # ADD CODE HERE
        sorted_indices = np.argsort(rf_clf.feature_importances_)[::-1]
        # -------------------------------
        return sorted_indices
    
# Q3.3.2 Hyper-parameter Tuning

    # points [2]
    def hyperParameterTuning(self,rf_clf,x_train,y_train):
        # TODO: Tune the hyper-parameters 'n_estimators' and 'max_depth'.
        # args: RandomForestClassifier object, pandas dataframe, pandas series
        # return: GridSearchCV object
        # 'n_estimators': [4, 16, 256]
        # 'max_depth': [2, 8, 16]
        # -------------------------------
        # ADD CODE HERE
        param_grid = {'n_estimators': [4, 16, 256], 'max_depth': [2, 8, 16]}
        gscv_rfc = GridSearchCV(estimator = rf_clf, param_grid = param_grid)
        gscv_rfc.fit(x_train, y_train)
        # -------------------------------
        return gscv_rfc
    
    # points [1]
    def bestParams(self,gscv_rfc):
        # TODO: Get the best params, using .best_params_
        # args:  GridSearchCV object
        # return: parameter dict
        # -------------------------------
        # ADD CODE HERE
        best_params = gscv_rfc.best_params_
        # -------------------------------
        return best_params
    
    # points [1]
    def bestScore(self,gscv_rfc):
        # TODO: Get the best score, using .best_score_.
        # args: GridSearchCV object
        # return: float
        # -------------------------------
        # ADD CODE HERE
        best_score = gscv_rfc.best_score_
        # -------------------------------
        return best_score

In [52]:
rm = RFClassifier()

In [54]:
rf_clf,y_predict_train, y_predict_test = rm.randomForestClassifier(x_train,x_test, y_train)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [55]:
train_accuracy = rm.rfTrainAccuracy(y_train,y_predict_train)

In [56]:
train_accuracy

0.9890577507598785

In [59]:
test_accuracy = rm.rfTestAccuracy(y_test,y_predict_test)

In [60]:
test_accuracy

0.5070821529745042

In [61]:
gscv_rfc = rm.hyperParameterTuning(rf_clf,x_train,y_train)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [64]:
best_params = gscv_rfc.best_params_

In [65]:
best_params

{'max_depth': 2, 'n_estimators': 256}

In [66]:
best_score = gscv_rfc.best_score_

In [67]:
bestScore

0.5458966565349544

In [68]:
#export
class SupportVectorMachine():
    
# Q3.4.1 Pre-process

    # points [1]
    def dataPreProcess(self,x_train,x_test):
        # TODO: Pre-process the data to standardize it, otherwise the grid search will take much longer.
        # args: pandas dataframe, pandas dataframe
        # return: pandas dataframe, pandas dataframe
        # -------------------------------
        # ADD CODE HERE
        scaler = StandardScaler()
        scaler.fit(x_train)
        scaled_x_train = scaler.transform(x_train)
        scaled_x_test = scaler.transform(x_test)
        # -------------------------------
        return scaled_x_train, scaled_x_test
    
# Q3.4.2 Classification

    # points [1]
    def SVCClassifier(self,scaled_x_train,scaled_x_test, y_train):
        # TODO: Create a SVC classifier and train it. Set gamma = 'auto'
        # args: pandas dataframe, pandas dataframe, pandas series
        # return: numpy array, numpy array
        # -------------------------------
        # ADD CODE HERE
        svm = SVC(gamma = 'auto')
        svm.fit(scaled_x_train, y_train)
        y_predict_train = svm.predict(scaled_x_train)
        y_predict_test = svm.predict(scaled_x_test)
        # -------------------------------
        return y_predict_train,y_predict_test
    
    # points [1]
    def SVCTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy on the training set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float 
        # -------------------------------
        # ADD CODE HERE
        y_train_pred_round = [round(k) for k in y_predict_train]
        train_accuracy = accuracy_score(y_train_pred_round, y_train)
        # -------------------------------
        return train_accuracy
    
    # points [1]
    def SVCTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy on the test set using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float 
        # -------------------------------
        # ADD CODE HERE
        y_test_pred_round = [round(k) for k in y_predict_test]
        test_accuracy = accuracy_score(y_test_pred_round, y_test)        
        # -------------------------------
        return test_accuracy
    
# Q3.4.3 Hyper-parameter Tuning
    
    # points [1]
    def SVMBestScore(self, scaled_x_train, y_train):
        # TODO: Tune the hyper-parameters 'C' and 'kernel' (use rbf and linear).
        # Note: Set n_jobs = -1 and , return_train_score=Ture and gamma = 'auto'
        # args: pandas dataframe, pandas series
        # return: GridSearchCV object, float
        # -------------------------------
        svm_parameters = {'kernel':('linear', 'rbf'), 'C':[0.01, 0.1, 1.0]}
        # ADD CODE HERE
        svm = SVC(gamma = 'auto')
        #svm.fit(scaled_x_train, y_train)
        svm_cv = GridSearchCV(estimator = svm, param_grid = svm_parameters, n_jobs = -1, return_train_score=True)
        svm_cv.fit(scaled_x_train, y_train)
        best_score = svm_cv.best_score_
        # -------------------------------
        
        return svm_cv, best_score
    
    # points [1]
    def SVCClassifierParam(self,svm_cv,scaled_x_train,scaled_x_test,y_train):
        # TODO: Calculate the training and test set accuracy values after hyperparameter tuning and standardization. 
        # args: GridSearchCV object, pandas dataframe, pandas dataframe, pandas series
        # return: numpy series, numpy series
        # -------------------------------
        # ADD CODE HERE
        #svm_cv, best_score = SVMBestScore(scaled_x_train, y_train)
        y_predict_train = svm_cv.predict(scaled_x_train)
        y_predict_test = svm_cv.predict(scaled_x_test)
        # -------------------------------
        return y_predict_train, y_predict_test

    # points [1]
    def svcTrainAccuracy(self,y_train,y_predict_train):
        # TODO: Return accuracy (on the training set) using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        train_accuracy = accuracy_score(y_predict_train, y_train)
        # -------------------------------
        return train_accuracy

    # points [1]
    def svcTestAccuracy(self,y_test,y_predict_test):
        # TODO: Return accuracy (on the test set) using the accuracy_score method.
        # args: pandas series, numpy array
        # return: float
        # -------------------------------
        # ADD CODE HERE
        test_accuracy = accuracy_score(y_predict_test, y_test)
        # -------------------------------
        return test_accuracy
    
# Q3.4.4 Cross Validation Results

    # points [1]
    def SVMRankTestScore(self,svm_cv):
        # TODO: Return the rank test score for all hyperparameter values that you obtained in Q3.4.3. The 
        # GridSearchCV class holds a 'cv_results_' dictionary that should help you report these metrics easily.
        # args: GridSearchCV object 
        # return: int array
        # -------------------------------
        # ADD CODE HERE
        """for i in range(len(svm_cv.cv_results_["params"])):
            if svm_cv.cv_results_["params"][i] == svm_cv.best_params_:
                rank_test_score = svm_cv.cv_results_["rank_test_score"][i]
        """
        rank_test_score = svm_cv.cv_results_["rank_test_score"]
        # -------------------------------
        return rank_test_score
    
    # points [1]
    def SVMMeanTestScore(self,svm_cv):
        # TODO: Return mean test score for all of hyperparameter values that you obtained in Q3.4.3. The 
        # GridSearchCV class holds a 'cv_results_' dictionary that should help you report these metrics easily.
        # args: GridSearchCV object
        # return: float array
        # -------------------------------
        # ADD CODE HERE
        """for i in range(len(svm_cv.cv_results_["params"])):
            if svm_cv.cv_results_["params"][i] == svm_cv.best_params_:
                mean_test_score = svm_cv.cv_results_["mean_test_score"][i]
        """
        mean_test_score = svm_cv.cv_results_["mean_test_score"]
        # -------------------------------
        return mean_test_score

In [69]:
svm = SupportVectorMachine()

In [70]:
scaled_x_train, scaled_x_test = svm.dataPreProcess(x_train,x_test)
y_predict_train,y_predict_test = svm.SVCClassifier(scaled_x_train,scaled_x_test, y_train)
train_accuracy = svm.SVCTrainAccuracy(y_train,y_predict_train)
test_accuracy = svm.SVCTestAccuracy(y_test,y_predict_test)
 


In [71]:
print("train_accuracy = " + str(train_accuracy))
print("test_accuracy = " + str(test_accuracy))

train_accuracy = 0.5610942249240122
test_accuracy = 0.5339943342776204


In [72]:
svm_cv, best_score = svm.SVMBestScore(scaled_x_train, y_train)
y_predict_train, y_predict_test = svm.SVCClassifierParam(svm_cv,scaled_x_train,scaled_x_test,y_train)
train_accuracy = svm.svcTrainAccuracy(y_train,y_predict_train)
test_accuracy = svm.svcTestAccuracy(y_test,y_predict_test)

/Users/jonhuaqin/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [73]:
print("train_accuracy = " + str(train_accuracy))
print("test_accuracy = " + str(test_accuracy))

train_accuracy = 0.5519756838905775
test_accuracy = 0.5311614730878187


In [76]:
rank_test_score = svm_cv.cv_results_["rank_test_score"]
mean_test_score = svm_cv.cv_results_["mean_test_score"]


In [77]:
print(rank_test_score)


[1 1 1 1 1 6]


In [78]:
print(mean_test_score)

[0.55197568 0.55197568 0.55197568 0.55197568 0.55197568 0.54346505]


In [79]:
temp = post[post['created_utc'] == '2014-12-31']
temp1 = temp[temp['submissionID'] == '2qzjqr']

In [80]:
temp1

,created_utc,submissionID,full_link,author,author_flair_richtext,num_comments,score,title,selftext,link
0,2014-12-31,2qzjqr,,livebitcoinnews,,0,1,"CheapAir now Taking over Bitcoin, Dogecoin and...",,https://www.reddit.com/r/Bitcoin/comments/2qzj...
88035,2014-12-31,2qzjqr,,livebitcoinnews,,0,1,"CheapAir now Taking over Bitcoin, Dogecoin and...",,https://www.reddit.com/r/Bitcoin/comments/2qzj...


In [83]:
print(temp1['link'][0])

https://www.reddit.com/r/Bitcoin/comments/2qzjqr/cheapair_now_taking_over_bitcoin_dogecoin_and/
